In [1]:

!pip -q install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

!pip -q install datasets transformers trl peft bitsandbytes accelerate scikit-learn pandas


import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import random, numpy as np, gc, hashlib
import torch
from typing import List
from datasets import load_dataset
from unsloth import FastLanguageModel
from peft import PeftModel  
from transformers import LogitsProcessor, TrainingArguments #  LogitsProcessor
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
import pandas as pd


SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

print("finish")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 29.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.6/273.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 34.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━

2025-11-01 23:38:42.894260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762040323.256026      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762040323.362750      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
finish


In [2]:

ADAPTER_PATH = "/kaggle/input/my-trained-adapters-v2"#/kaggle/input/my-trained-adapters-v2  /kaggle/input/my-trained-adapters-v2


MAX_LEN_GEN  = 640  
dtype = None
load_in_4bit = True

# check device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
print(f"add from: {ADAPTER_PATH}")

Device: cuda
add from: /kaggle/input/my-trained-adapters-v2


In [3]:


print("from Hugging Face  'unsloth/Meta-Llama-3.1-8B' (4-bit)...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name   = "unsloth/Meta-Llama-3.1-8B", 
    max_seq_length = MAX_LEN_GEN,
    dtype        = dtype,
    load_in_4bit = load_in_4bit,
)

print("\nLora...")
try:
    
    model = PeftModel.from_pretrained(model, ADAPTER_PATH)
    print("adapter ready！")
except Exception as e:
    print(f"\n{'='*50}\n '{ADAPTER_PATH}' right？\n not: {e}\n{'='*50}")
    
print("\n (Unsloth)...")

FastLanguageModel.for_inference(model) 
print("model ready")

# ---  Tokenizer ---
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"   
tokenizer.truncation_side = "left" 
try:
    model.config.pad_token_id = tokenizer.pad_token_id
except Exception:
    pass

# ---  Token IDs ---
true_ids  = tokenizer.encode(" True", add_special_tokens=False)
false_ids = tokenizer.encode(" False", add_special_tokens=False)
allowed_first_tokens = list({ true_ids[0], false_ids[0] })
print(f"True IDs: {true_ids}, False IDs: {false_ids}")

gc.collect(); torch.cuda.empty_cache() if torch.cuda.is_available() else None

from Hugging Face  'unsloth/Meta-Llama-3.1-8B' (4-bit)...
==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]


Lora...


/usr/local/lib/python3.11/dist-packages/peft/config.py:165: UserWarning: Unexpected keyword arguments ['target_parameters'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


adapter ready！

 (Unsloth)...
model ready
True IDs: [3082], False IDs: [3641]


In [4]:

print("from Hugging Face ..")

full = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

def qhash(q):
    return int(hashlib.md5(q.encode("utf-8")).hexdigest(), 16) % (10**9)

groups = [qhash(q) for q in full["question"]]
rng = np.random.default_rng(SEED)
unique_groups = np.array(sorted(set(groups)))
rng.shuffle(unique_groups)
cut = int(len(unique_groups) * 0.9)
train_group_set = set(unique_groups[:cut])
valid_idx = [i for i, g in enumerate(groups) if g not in train_group_set]
valid_ds = full.select(valid_idx)
y_true_valid = [int(x) for x in valid_ds["is_correct"]] 

test_ds = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")

PROMPTS = [
    "Q:\n{}\nA:\n{}\nOutput:\n",
    "Question:\n{}\nStudent Answer:\n{}\nOutput:\n",
    "Problem:\n{}\nSolution:\n{}\nOutput:\n",
]


def fmt_infer(b, tpl):
    qs, ss = b["question"], b["solution"]
    return {"text": [tpl.format(q, str(s)) for q, s in zip(qs, ss)]}

valid_formatted_list = []
for tpl in PROMPTS:
    valid_formatted_list.append(
        valid_ds.map(lambda b: fmt_infer(b, tpl), batched=True, remove_columns=valid_ds.column_names)
    )

test_formatted_list = []
for tpl in PROMPTS:
    test_formatted_list.append(
        test_ds.map(lambda b: fmt_infer(b, tpl), batched=True, remove_columns=test_ds.column_names)
    )

print(f"valid: {len(valid_ds)}, test: {len(test_ds)}")

from Hugging Face ..


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

...

Map:   0%|          | 0/98474 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

valid: 98474, test: 10000


In [5]:

class OnlyAllowTF(LogitsProcessor):
    def __init__(self, step:int): self.step = step
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        if self.step == 0:
            mask = torch.full_like(scores, float("-inf"))
            mask[:, allowed_first_tokens] = 0.0
            scores = scores + mask
        return scores


def autotune_chunk(texts, start=16, floor=2, ceil=32):
    import math
    chunk = min(start, max(floor, len(texts)))
    print(f"find optimal CHUNK， start={chunk}...")
    while chunk >= floor:
        try:
            probe = texts[:min(len(texts), chunk)]
            _inputs = tokenizer(
                probe, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LEN_GEN
            ).to(model.device)
            with torch.no_grad():
                _ = model.generate(
                    **_inputs,
                    max_new_tokens=2, do_sample=False, temperature=0.0, top_p=1.0,
                    pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id,
                    logits_processor=[OnlyAllowTF(step=0)], use_cache=True,
                )
            del _inputs, _
            if torch.cuda.is_available(): torch.cuda.empty_cache()
            print(f" CHUNK = {chunk}")
            return min(chunk, ceil) 
        except RuntimeError as e:
            if "CUDA out of memory" in str(e):
                print(f"    OOM CHUNK = {chunk}")
                if torch.cuda.is_available(): torch.cuda.empty_cache()
                chunk //= 2 
            else:
                print(f"error: {e}")
                raise
    print(f"  {floor} return {floor}")
    return floor


def generate_TF(batch_texts: List[str], chunk: int = 4) -> List[int]:
    outs = []
    CHUNK = chunk
    for i in range(0, len(batch_texts), CHUNK):
        chunk_texts = batch_texts[i:i+CHUNK]
        inputs = tokenizer(
            chunk_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=MAX_LEN_GEN,
        ).to(model.device)
        with torch.no_grad():
            gen = model.generate(
                **inputs,
                max_new_tokens=2,
                do_sample=False,
                temperature=0.0,
                top_p=1.0,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                logits_processor=[OnlyAllowTF(step=0)],
                use_cache=True,
            )
        decoded = tokenizer.batch_decode(gen, skip_special_tokens=True)
        tails = [full[len(prompt):].strip() if full.startswith(prompt) else full
                 for full, prompt in zip(decoded, chunk_texts)]
        
       
        outs.extend([1 if t.lower().lstrip().startswith("true") else 0 for t in tails])
        
        del inputs, gen, decoded
        if torch.cuda.is_available(): torch.cuda.empty_cache()
    return outs


def generate_vote(ds, formatted_list, chunk=4):
    import numpy as np
    votes = np.zeros(len(ds), dtype=int)
    for i, tpl in enumerate(PROMPTS):
        print(f"  Prompt {i+1}/{len(PROMPTS)} ('{tpl.splitlines()[0]}...')")
        texts = formatted_list[i]["text"]
        preds = generate_TF(texts, chunk=chunk)
        votes += np.array(preds)
    
    final_preds = (votes >= ((len(PROMPTS) + 1)//2)).astype(int)
    return final_preds

print("vote function")

vote function


In [ ]:

probe_texts = valid_formatted_list[0]["text"]
CHUNK_VALID = autotune_chunk(probe_texts, start=16, ceil=32) 
print("===== run test =====")

CHUNK_VALID
CHUNK_TEST = CHUNK_VALID
print(f"===== [Autotune] Test CHUNK = {CHUNK_TEST} =====")


test_preds = generate_vote(test_ds, test_formatted_list, chunk=CHUNK_TEST)


print("\n create submission.csv...")
submission = pd.DataFrame({
    "ID": range(len(test_preds)),
    "is_correct": test_preds,
})

out_path = "submission.csv"
submission.to_csv(out_path, index=False)

print(f"\n===== finish=====")
print(f"saved to: {out_path}")
print(submission.head())

find optimal CHUNK， start=16...


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


 CHUNK = 16
===== run test =====
===== [Autotune] Test CHUNK = 16 =====
  Prompt 1/3 ('Q:...')


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
